# StateMachine II

In [ ]:
from wxyz.stm.widget_stm import StateMachine, W
from wxyz.lab.widget_dock import DockBox
from yaml import safe_load

In [ ]:
with __import__("importnb").Notebook():
    from StateMachine import make_an_adventure

In [ ]:
if __name__ == "__main__":
    state_dict = safe_load("""
    jail:
        _:
            find_key: jail_has_key
            sleep: jail
        has_key:
            _:
                escape: town
    town:
        _:
            steal_get_caught: jail
            steal: town_has_coin
            beg: town_has_coin
        has_coin:
            _:
                buy_sword: town_has_gear_sword
                buy_wand: town_has_gear_wand
        has_gear:
            _:
                practice: town_has_gear
            sword:
            wand:
    """)
    display(state_dict)

In [ ]:
def state_dict_to_state_transitions(state_dict, prefix="", parents=[]):
    states = []
    transitions = []
    for state, children in state_dict.items():
        if state == "_":
            for tx, tgt in children.items():
                if prefix:
                    transitions += [
                        [tx, parents[-1], tgt]
                    ]
                else:
                    transitions += [
                        [tx, state, tgt]
                    ]
        else:
            cs, ct = [], []
            if children:
                cs, ct = state_dict_to_state_transitions(children, f"{prefix}{state}_", [*parents, f"{prefix}{state}"])
            if cs:
                states += [
                    {"name": state, "children": cs}
                ]
            else:
                states += [state]
            transitions += ct
    return states, transitions

In [ ]:
if __name__ == "__main__":
    states, transitions = state_dict_to_state_transitions(state_dict)
    display(states, transitions)

In [ ]:
if __name__ == "__main__":
    stm, view = make_an_adventure(states=states, transitions=transitions, initial="jail")
    dock = DockBox(view.children, layout=dict(height="600px"))

    @dock.on_displayed
    def _on_displayed(_):
        dock.dock_layout = {
            'type': 'split-area',
            'orientation': 'horizontal',
            'children': [
                {'type': 'tab-area', 'widgets': [0], 'currentIndex': 0},
                {'type': 'tab-area', 'widgets': [1], 'currentIndex': 0}
            ], 'sizes': [0.3, 0.7]}
    display(dock)